<a href="https://colab.research.google.com/github/gabrielanatalia/PortfolioVisualizer/blob/main/Portfolio_Visualizer_colab_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Portfolio Visualizer**



In [8]:
# @title 1. Load libraries and functions
!pip install reportlab
!pip install kaleido
!pip install PyPDF2

import pandas as pd
import numpy as np
import math, os, io, time, pytz, re, copy
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, timedelta
import statsmodels.api as sm
from scipy.stats import norm
from scipy.optimize import minimize, fsolve
import itertools
import plotly.graph_objects as go
import yfinance as yf
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
from matplotlib import colors
import statsmodels.formula.api as smf
from dateutil.parser import parse
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Javascript
from ipywidgets import VBox, HBox, Label, HTML
from google.colab import widgets as gc_widgets
from contextlib import redirect_stdout
from google.colab import files
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from IPython.display import FileLink
from matplotlib.backends.backend_pdf import PdfPages
import PyPDF2
from PyPDF2 import PdfMerger, PdfReader, PdfWriter


!git clone https://github.com/gabrielanatalia/Projects/
%cd /content/Projects
import sys
sys.path.append('/content/Projects')
import port_cons as pc

import warnings
warnings.filterwarnings("ignore")

def convert_to_datetime(input_str, parserinfo=None):
    return parse(input_str, parserinfo=parserinfo)

TOLERANCE = 1e-10

def _allocation_risk(weights, covariances):

    portfolio_risk = np.sqrt((weights * covariances * weights.T))[0, 0]

    return portfolio_risk

def _assets_risk_contribution_to_allocation_risk(weights, covariances):

    portfolio_risk = _allocation_risk(weights, covariances)

    assets_risk_contribution = np.multiply(weights.T, covariances * weights.T) \
        / portfolio_risk

    return assets_risk_contribution

def _risk_budget_objective_error(weights, args):
    covariances = args[0]
    assets_risk_budget = args[1]
    weights = np.matrix(weights)

    portfolio_risk = _allocation_risk(weights, covariances)

    assets_risk_contribution = \
        _assets_risk_contribution_to_allocation_risk(weights, covariances)

    assets_risk_target = \
        np.asmatrix(np.multiply(portfolio_risk, assets_risk_budget))

    error = sum(np.absolute(assets_risk_contribution - assets_risk_target.T))[0, 0]
    return error

def _get_risk_parity_weights(covariances, assets_risk_budget, initial_weights):

    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0},{'type': 'ineq', 'fun': lambda x: x})

    optimize_result = minimize(fun=_risk_budget_objective_error,
                               x0=initial_weights,
                               args=[covariances, assets_risk_budget],
                               method='SLSQP',
                               constraints=constraints,
                               tol=TOLERANCE,
                               options={'disp': False})

    weights = optimize_result.x
    print(optimize_result.message)
    return weights

def rebal_wgt_riskparity(returns_data, start_date, end_date, rebal_months=[4,10], halflife=3.5, annualized=252, shrink_covar=False):
    num_of_assets = len(returns_data.columns)

    ret_data_filtered = returns_data.loc[start_date:end_date]
    first_date = returns_data.index[0]
    tickers = list(returns_data.columns)

    weights = pd.DataFrame(0, index=ret_data_filtered.index, columns=ret_data_filtered.columns)
    weights.index = pd.to_datetime(weights.index, format='%Y-%m-%d')
    reb_flag = pd.DataFrame(0, index=weights.index, columns=['reb_flag'])

    alpha = 1 - math.exp(math.log(0.5) / (halflife * annualized))
    span = (2 / alpha) - 1

    for i in range(len(ret_data_filtered)):
        curr_date = ret_data_filtered.index[i]
        curr_date_format = curr_date.strftime('%Y-%m-%d')

        if (weights.index[i].month in rebal_months and weights.index[i-1].month != weights.index[i].month) or i==0:

            if shrink_covar:
                shrink_covar_matrix_model = LedoitWolf().fit(returns_data.loc[first_date:curr_date])
                covar_ann = pd.DataFrame(shrink_covar_matrix_model.covariance_ * annualized, index=tickers, columns=tickers)
                covar_ann.columns = tickers
                covar_ann.index = tickers
            else:
                exp_cov_matrix = returns_data.loc[first_date:curr_date].ewm(span=span).cov(pairwise=True).iloc[-num_of_assets:]
                covar_ann = exp_cov_matrix * annualized
                covar_ann.columns = tickers
                covar_ann.index = tickers

            valid_covar_ann = covar_ann.dropna(how='all')
            valid_covar_ann = valid_covar_ann.dropna(axis=1, how='all')
            valid_num_assets = len(valid_covar_ann)
            valid_tickers = valid_covar_ann.columns

            assets_risk_budget = np.ones([valid_num_assets]) / valid_num_assets
            initial_weights = np.ones([valid_num_assets]) / valid_num_assets

            # print(valid_covar_ann)
            new_wgt = _get_risk_parity_weights(valid_covar_ann.values, assets_risk_budget, initial_weights)
            new_wgt = pd.DataFrame(new_wgt).T
            new_wgt.columns = valid_tickers

            for ticker in valid_tickers:
                weights.at[curr_date_format, ticker] = new_wgt[ticker].values

            reb_flag.loc[curr_date_format] = True

        else:
            weights.iloc[i] = weights.iloc[i-1] * (1+ ret_data_filtered.iloc[i].fillna(0))
            weights_sum = weights.iloc[i].sum()
            weights.iloc[i] /= weights_sum
            reb_flag.iloc[i] = False

    weights = weights.rename(columns={c: c + '_wgt' for c in weights.columns})
    weights = pd.concat([reb_flag, weights], axis=1)

    return weights

def combine_backtest_data(portfolio_names):
    # combined portfolio returns
    all_port_ret = {}

    for portfolio in portfolio_names:
        perf_df = globals()[f"{portfolio}_perf"]
        ret_col = [col for col in perf_df.columns if col.endswith('_port_ret')]
        all_port_ret[portfolio] = perf_df[ret_col]

    df_all_port_ret = pd.concat(all_port_ret.values(), keys=all_port_ret.keys(), axis=1)
    df_all_port_ret.columns = [col[0] for col in df_all_port_ret.columns]

    # combined portfolio weights
    all_port_weights = {}

    for portfolio in portfolio_names:
        wgt_df = globals()[f"{portfolio}_wgt"]
        # wgt_col = [col for col in wgt_df.columns if col.endswith('_wgt')]
        # all_port_weights[portfolio] = wgt_df[wgt_col]
        all_port_weights[portfolio] = wgt_df

    df_all_port_weights = pd.concat(all_port_weights.values(), keys=all_port_weights.keys(), axis=1)

    # combined portfolio backtest daata
    all_port_bt = {}

    for portfolio in portfolio_names:
        bt_df = globals()[f"{portfolio}_perf"]
        all_port_bt[portfolio] = bt_df

    df_all_port_bt = pd.concat(all_port_bt.values(), keys=all_port_bt.keys(), axis=1)
    return df_all_port_ret, df_all_port_weights, df_all_port_bt


def print_arial(text):
    display(HTML(f"<div style='font-family: Arial, sans-serif'>{text}</div>"))

def print_arial_bold(text):
    display(HTML(f"<div style='font-family: Arial, sans-serif; font-weight: bold;'>{text}</div>"))

def merge_pdfs(existing_pdf, merged_pdf, output_pdf):
    # Open the existing PDFs
    with open(existing_pdf, "rb") as pdf1, open(merged_pdf, "rb") as pdf2:
        reader1 = PyPDF2.PdfReader(pdf1)
        reader2 = PyPDF2.PdfReader(pdf2)
        writer = PyPDF2.PdfWriter()

        # Add all pages from the first PDF
        for page in reader1.pages:
            writer.add_page(page)

        # Add all pages from the second PDF
        for page in reader2.pages:
            writer.add_page(page)

        # Save the merged PDF
        with open(output_pdf, "wb") as output:
            writer.write(output)

def plot_pie_chart_topX(series, title, topX=10, width=1000, height=600, colors=None):
    # Filter for values greater than 0
    series = series[series > 0]

    # Proceed only if there are values to plot
    if not series.empty:
        top_x = series.sort_values(ascending=False).head(topX)
        other_sum = series.sum() - top_x.sum()

        # Filter labels and values based on positive values
        labels = list(top_x.index)
        values = list(top_x)

        # Add 'Others' only if other_sum is greater than 0
        if other_sum > 0.01:
            labels.append('Others')
            values.append(other_sum)

        # Create the Pie trace data
        pie_data = {
            'labels': labels,
            'values': values,
            'textinfo': 'label+percent',
            'hole': 0.3,
            'textposition': 'outside',
        }

        # Add marker with custom colors if provided
        if colors:
            pie_data['marker'] = dict(colors=colors)

        fig = go.Figure(data=[go.Pie(**pie_data)])  # Unpack pie_data into go.Pie
        fig.update_layout(title={'text': f'<b>{title}</b>','font': dict(size=75)}, width=width, height=height, showlegend=False, font=dict(family="Arial", size=75, color='black'))
        return fig
    else:
        print("No values greater than 0 to plot.")
        return None

def plot_bar_chart_topX(series, title, topX=10, width=1000, height=600, colors=None):


    # Filter for values greater than 0
    series = series[series > 0]

    # Proceed only if there are values to plot
    if not series.empty:
        top_x = series.sort_values(ascending=False).head(topX)
        other_sum = series.sum() - top_x.sum()

        # Filter labels and values based on positive values
        labels = list(top_x.index)
        values = list(top_x)

        # Add 'Others' only if other_sum is greater than 0
        if other_sum > 0:
            labels.append('Others')
            values.append(other_sum)

        # Calculate percentages and format labels
        total_sum = sum(values)
        text_labels = [f'{round(value / total_sum * 100, 2)}%' for value in values]  # Only percentage

        # Create the Bar trace data
        bar_data = {
            'x': labels,
            'y': values,
            'text': text_labels,
            'textposition': 'auto',
            'textangle': 0
        }

        # Add marker with the first custom color if provided
        if colors:
            bar_data['marker'] = dict(color=colors[0])  # Use the first color from the list

        fig = go.Figure(data=[go.Bar(**bar_data)])  # Unpack bar_data into go.Bar
        fig.update_layout(title={'text': f'<b>{title}</b>','font': dict(size=75)}, width=width, height=height, showlegend=False, yaxis_tickformat=".0%", font=dict(family="Arial", size=75, color='black'))
        return fig
    else:
        print("No values greater than 0 to plot.")
        return None

def calc_custom_weighted_risk_rating(custom_usd_rows, df_sec_list):
    # Calculates the weighted risk rating of custom USD portfolios

    # Merge risk ratings from security list
    custom_usd_risk_rating = custom_usd_rows.copy()
    custom_usd_risk_rating = custom_usd_risk_rating.merge(
        df_sec_list[['Ticker', 'Risk rating']], how='left', left_on='type', right_on='Ticker'
    )

    # Calculate weighted risk rating and NAV in SGD
    custom_usd_risk_rating_weighted_avg = custom_usd_risk_rating.groupby('portfolio_id').apply(
        lambda x: pd.Series({
            'Risk rating': np.average(x['Risk rating'], weights=x['nav_in_sgd']),
        })
    ).reset_index()

    return custom_usd_risk_rating_weighted_avg

########################################################################################################################

# read asseet class, sector, country data from csv
data_path = '/content/Projects/Data/'
df_sec_list = pd.read_csv(data_path + 'PV_sec_list.csv')
df_sec_list['Remarks'] = df_sec_list['Remarks'].fillna('N/A')
df_asset_class = pd.read_csv(data_path + 'PV_asset_class.csv', index_col=0)
df_sector = pd.read_csv(data_path + 'PV_sector.csv', index_col=0)
df_country = pd.read_csv(data_path + 'PV_country.csv', index_col=0)
df_fi_metrics = pd.read_csv(data_path + 'PV_FI_metrics.csv', index_col=0)
df_fx = pd.read_csv(data_path + 'PV_daily_ret.csv',header=0, index_col='Date', parse_dates=['Date'],dayfirst=True)[['USDSGD', 'USDHKD']]/100


syfe_colors = ['#263159', '#2f51c9', '#879be3', '#bcbed7', '#dedfee','#fff2cc', '#ffe599', '#e3bf61', '#666666', '#7d839b', '#414e7d']

Cloning into 'Projects'...
remote: Enumerating objects: 558, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 558 (delta 68), reused 42 (delta 42), pack-reused 461 (from 1)
Receiving objects: 100% (558/558), 40.31 MiB | 15.56 MiB/s, done.
Resolving deltas: 100% (328/328), done.
/content/Projects


In [39]:
# @title PDF Funcs
from reportlab.lib.styles import ParagraphStyle
from reportlab.lib.pagesizes import landscape, A4
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Spacer, Image, Paragraph
import os


def create_table(df, title):

    elements = []
    if df.empty:
        return elements
    else:
        table_data = [df.columns.tolist()]  # Add header row
        for _, row in df.iterrows():
            table_data.append(list(row))

        table = Table(table_data)
        table_style = TableStyle(
            [
                ("BACKGROUND", (0, 0), (-1, 0), colors.HexColor("#263159")),
                ("TEXTCOLOR", (0, 0), (-1, 0), colors.whitesmoke),
                ("ALIGN", (0, 0), (-1, -1), "CENTER"),
                ("FONTNAME", (0, 0), (-1, 0), "Helvetica-Bold"),
                ("FONTSIZE", (0, 0), (-1, 0), 9),
                ("BOTTOMPADDING", (0, 0), (-1, 0), 3),
                ("TOPPADDING", (0, 0), (-1, 0), 3),
                ("BOTTOMPADDING", (0, 1), (-1, -1), 1),
                ("TOPPADDING", (0, 1), (-1, -1), 1),
                ("BACKGROUND", (0, 1), (-1, -1), colors.whitesmoke),
                ("TEXTCOLOR", (0, 1), (-1, -1), colors.black),
                ("FONTNAME", (0, 1), (-1, -1), "Helvetica"),
                ("FONTSIZE", (0, 1), (-1, -1), 8),
                ("VALIGN", (0, 0), (-1, -1), "TOP"),
                ("ROWBACKGROUNDS", (0, 1), (-1, -1), [colors.whitesmoke, colors.lightgrey]),
                ("FONTNAME", (0, -1), (-1, -1), "Helvetica-Bold"),
            ]
        )

        for i in range(len(df)):  # Iterate using numerical index
              if df.iloc[i]['Portfolio ID'] in ['Growth Portfolios', 'Income & Preservation Portfolios', "Core", "Specialised", "Income+", "REITs", "Alternatives","Cash / MMF"]:
                  table_style.add("BACKGROUND", (0, i + 1), (-1, i + 1), colors.HexColor("#879be4"))

        table.setStyle(table_style)

        custom_title_style = ParagraphStyle(
            name="CustomTitle",
            fontName="Helvetica-Bold",
            fontSize=12,
            leading=12,  # Line height
            alignment=0,  # Left align
        )

        # Update the title line in your function
        elements.append(Spacer(1, 0.5 * inch))
        elements.append(Paragraph(f"{title}", custom_title_style))
        elements.append(Spacer(1, 0.2 * inch))  # Reduced space after the title
        elements.append(table)

    return elements

def add_page_numbers_header(input_pdf, output_pdf, client_name, page_size=landscape(A4)):
    page_width, page_height = page_size

    reader = PdfReader(input_pdf)
    writer = PdfWriter()
    total_pages = len(reader.pages)

    for page_num, page in enumerate(reader.pages, start=1):
        packet = io.BytesIO()
        can = canvas.Canvas(packet, pagesize=page_size)
        can.setFont("Helvetica", 8)

        # Add page number to bottom-right corner
        page_number_text = f"Page {page_num} of {total_pages}"
        text_width = can.stringWidth(page_number_text, "Helvetica", 8)
        x_position = page_width - 20 - text_width
        y_position = 10
        can.drawString(x_position, y_position, page_number_text)

        # Add header to upper right-hand corner (same as before)
        header_x = page_width - 20  # Start from the right edge

        # Client Name
        client_name_text = f"Client Name: {client_name}"
        client_name_width = can.stringWidth(client_name_text, "Helvetica", 8)
        can.drawString(header_x - client_name_width, page_height - 20, client_name_text)  # Calculate x for right alignment

        # Date
        date_text = f"Date: {datetime.now().strftime('%B %d, %Y')}"
        date_width = can.stringWidth(date_text, "Helvetica", 8)
        can.drawString(header_x - date_width, page_height - 30, date_text)  # Calculate x for right alignment

        can.save()

        packet.seek(0)
        page_overlay = PdfReader(packet)
        page.merge_page(page_overlay.pages[0])
        writer.add_page(page)

    with open(output_pdf, "wb") as f:
        writer.write(f)

def merge_with_template(template_pdf_path, overlay_pdf_path, output_pdf_path):
    """Merges multiple overlay pages onto a single-page template, repeating the template as needed."""

    template_pdf = PdfReader(template_pdf_path)
    overlay_pdf = PdfReader(overlay_pdf_path)
    output_pdf = PdfWriter()

    template_page = template_pdf.pages[0]  # Since the template has only 1 page

    for page_num in range(len(overlay_pdf.pages)):
        overlay_page = overlay_pdf.pages[page_num]

        # Copy the template so it doesn't get modified
        template_copy = template_page.create_blank_page(
            width=template_page.mediabox[2], height=template_page.mediabox[3]
        )
        template_copy.merge_page(template_page)  # Copy original template
        template_copy.merge_page(overlay_page)  # Overlay the new content

        output_pdf.add_page(template_copy)

    # Write final merged PDF
    with open(output_pdf_path, "wb") as final_pdf:
        output_pdf.write(final_pdf)

def create_pdf_with_table_and_charts(output_file, table_df, table_title, charts, col_width_manual=None):
    elements = []
    temp_images = []  # Track temporary image files for cleanup

    # Create PDF document
    doc = SimpleDocTemplate(
        output_file,
        pagesize=landscape(A4),
        leftMargin=0.4 * inch,  # Adjust left margin
        rightMargin=0.3 * inch,  # Adjust right margin
        topMargin=0.6 * inch,  # Adjust top margin
        bottomMargin=0.2 * inch  # Adjust bottom margin
    )

    # Calculate available width for table
    page_width = landscape(A4)[0]
    available_width = page_width - (doc.leftMargin + doc.rightMargin)

    # Create table elements
    table_elements = create_table(table_df, table_title)

    if table_elements:
        table = table_elements[-1]  # Last element is the table

        if isinstance(table, Table):
            # Calculate column widths dynamically based on text length
            text_lengths = table_df.applymap(lambda x: len(str(x))).sum(axis=0)
            total_text_length = sum(text_lengths)

            # Normalize column widths based on relative text length
            if col_width_manual is None:
                col_widths = [(available_width * (col_size / total_text_length)) for col_size in text_lengths]
                table._argW = col_widths  # Manually set column widths
            else:
                table._argW = col_width_manual
        # print(table._argW)
        elements.extend(table_elements)  # Add table elements

        # Add explanatory text if the table is for "Income & Preservation Portfolios"
        if table_title == "Income & Preservation Portfolios":
            explanation_text = (
                "<i>*Est. Yield (%): References Yield to Maturity (YTM) for Income+ portfolios, estimated dividend yield for REIT+ portfolios, projected returns for Cash+ Flexi portfolios, and guaranteed rates for Cash+ Guaranteed portfolios.</i><br/>"
                "<i>**Proj. Dividend: Estimated dividend yield multiplied by current portfolio value.</i>"
            )
            text_style = ParagraphStyle(
                name="ExplanationStyle",
                fontName="Helvetica-Oblique",  # Italic font
                fontSize=6,
                leading=8,  # Line spacing
                alignment=0,  # Left-aligned
            )
            elements.append(Spacer(1, 0.1 * inch))  # Space before text
            elements.append(Paragraph(explanation_text, text_style))

        elements.append(Spacer(1, 0.2 * inch))  # Space before charts

    # Chart settings
    num_charts = len(charts)
    cols_per_row = 2 if num_charts % 2 == 0 else 3  # Adjust layout

    chart_width = 3.5 * inch
    chart_height = 2.6 * inch

    chart_grid = []
    row = []

    for idx, chart in enumerate(charts):
        # Save chart to temporary image
        image_file = f"temp_chart_{idx}.png"
        chart.write_image(image_file, format="png", width=4000, height=3000)
        temp_images.append(image_file)

        # Add chart image to row
        img = Image(image_file, width=chart_width, height=chart_height)
        row.append(img)

        # Add row when cols_per_row charts are added or if it's the last chart
        if len(row) == cols_per_row or idx == len(charts) - 1:
            chart_grid.append(row)
            row = []

    # Create a table for charts with borders and adjusted colWidths
    for chart_row in chart_grid:
        col_widths = [available_width / len(chart_row)] * len(chart_row)  # Auto-fit chart columns
        chart_table = Table([chart_row], colWidths=col_widths)

        # Add border to chart table
        chart_table.setStyle(TableStyle([
            ('BOX', (0, 0), (-1, -1), 0.1, colors.white),
        ]))
        elements.append(chart_table)
        elements.append(Spacer(1, 0.1 * inch))

    doc.build(elements)

    # Clean up temporary image files
    for image_file in temp_images:
        if os.path.exists(image_file):
            os.remove(image_file)

def calculate_subtotal_income_preservation(df, classification):
    # Filter data for the given classification
    subset = df[df['L2_classification'] == classification]

    # Sum relevant numeric columns
    subtotal = subset[['Value (SGD)', 'Value (USD)', 'Proj. Dividend (SGD)', 'Allocation']].sum()

    # Assign Portfolio ID
    subtotal['Portfolio ID'] = classification

    # Calculate weighted average for Est. Yield (%)
    allocation_sum = subset['Allocation'].sum()

    if allocation_sum != 0:
        weighted_yield = (subset['Est. Yield (%)'] * subset['Allocation']).sum() / allocation_sum
    else:
        weighted_yield = 0  # Avoid division by zero

    # Add weighted Est. Yield (%) to the subtotal
    subtotal['Est. Yield (%)'] = weighted_yield

    return subtotal

def dividend_option_adj(row):
    if row['Dividend Option'] == 'REINVEST':
        return 'Accumulating'
    elif row['internal_port_name'] in income_plus_groups:
        return 'Payout (monthly)'
    elif row['internal_port_name'] in reit_groups:
        return 'Payout (quarterly)'
    else:
        return '-'

def format_maturity_period(value):
    if isinstance(value, str):  # Check if value is a string
        if value == '"ONE_MONTH"':
            return "1 month"
        if value == '"THREE_MONTHS"':
            return "3 months"
        if value == '"SIX_MONTHS"':
            return "6 months"
        if value == '"TWELVE_MONTHS"':
            return "12 months"
        else:
            return "-"
    return value  # Return original value if not a string or no match


In [40]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


syfe_port_mapping = pd.DataFrame({
'MB_port_name': ['CASH_PLUS', 'CASH_PLUS_USD', 'CHINA_GROWTH', 'CORE_BALANCED', 'CORE_DEFENSIVE', 'CORE_GROWTH', 'DISRUPTIVE_TECHNOLOGY', 'ESG_AND_CLEAN_ENERGY',
                  'GLOBAL_EQUITY_100', 'HEALTHCARE_INNOVATION', 'INCOME_ENHANCE', 'INCOME_PRESERVE', 'REIT', 'REIT_RISK_MANAGED',
                  'SRS_CASH_PLUS', 'SRS_GLOBAL_EQUITY_100', 'SRS_INCOME_ENHANCED', 'SRS_INCOME_PRESERVE', 'DOWNSIDE_PROTECTED', 'CASH_PLUS_GUARANTEED_SGD',
                 'CUSTOM_USD', 'SIFN27D'],

'internal_port_name': ['Cash SGD flexi', 'Cash USD flexi', 'China Growth', 'Core Balanced', 'Core Defensive', 'Core Growth', 'Disruptive Technology',
                        'ESG & Clean Energy', 'Core Equity100', 'Healthcare Innovation', 'Income Enhance', 'Income Preserve', 'REIT 100', 'REIT Risk Managed',
                        'SRS Cash SGD flexi', 'SRS Equity100', 'SRS Income Enhance', 'SRS Income Preserve', 'Downside protected', 'Cash SGD guaranteed',
                       'Custom USD', 'Silverdale Fund (Nov 2027)']
})

core_groups = ['Core Equity100', 'Core Growth', 'Core Balanced', 'Core Defensive', 'SRS Equity100']
specialised_groups = ['China Growth', 'Disruptive Technology', 'ESG & Clean Energy', 'Healthcare Innovation', 'Downside protected', 'Custom USD']
reit_groups = ['REIT 100', 'REIT Risk Managed']
income_plus_groups = ['Income Enhance', 'Income Preserve', 'SRS Income Enhance', 'SRS Income Preserve']
alt_groups = ['Silverdale Fund (Nov 2027)']
custom_groups = ['Custom USD']
cm_groups = ['Cash SGD flexi', 'SRS Cash SGD flexi', 'Cash USD flexi', 'Cash SGD guaranteed']

growth_groups = core_groups + specialised_groups
income_preservation_groups = income_plus_groups + reit_groups + alt_groups + cm_groups
all_syfe__portfolios = growth_groups + income_preservation_groups

syfe_port_mapping['L1_classification'] = np.where(syfe_port_mapping['internal_port_name'].isin(growth_groups), 'Growth', 'Income & Preservation')

L2_conditions = [
    syfe_port_mapping['internal_port_name'].isin(core_groups),
    syfe_port_mapping['internal_port_name'].isin(specialised_groups),
    syfe_port_mapping['internal_port_name'].isin(reit_groups),
    syfe_port_mapping['internal_port_name'].isin(income_plus_groups),
    syfe_port_mapping['internal_port_name'].isin(alt_groups),
    syfe_port_mapping['internal_port_name'].isin(cm_groups),
]

labels = ['Core', 'Specialised', 'REITs', 'Income+', 'Alternatives','Cash / MMF']

# Assign L2 classification
syfe_port_mapping['L2_classification'] = np.select(L2_conditions, labels, default='Other')

syfe_port_mapping = syfe_port_mapping.merge(df_sec_list[['Ticker', 'Risk rating']], how='left', left_on='internal_port_name', right_on='Ticker')
syfe_port_mapping.drop(columns=['Ticker'], inplace=True)


################################################################################################################################
mb_port_names_core = syfe_port_mapping[syfe_port_mapping['internal_port_name'].isin(core_groups)]['MB_port_name'].tolist()
mb_port_names_specialised = syfe_port_mapping[syfe_port_mapping['internal_port_name'].isin(specialised_groups)]['MB_port_name'].tolist()
mb_port_names_cm = syfe_port_mapping[syfe_port_mapping['internal_port_name'].isin(cm_groups)]['MB_port_name'].tolist()
mb_port_names_growth = syfe_port_mapping[syfe_port_mapping['internal_port_name'].isin(growth_groups)]['MB_port_name'].tolist()
mb_port_names_income_preservation = syfe_port_mapping[syfe_port_mapping['internal_port_name'].isin(income_preservation_groups)]['MB_port_name'].tolist()
mb_port_names_income_plus = ['INCOME_ENHANCE', 'INCOME_PRESERVE', 'SRS_INCOME_ENHANCE', 'SRS_INCOME_PRESERVE']
mb_port_names_reits = ['REIT', 'REIT_RISK_MANAGED']
mb_port_names_alt = ['SIFN27D']

order_of_ports = ['GLOBAL_EQUITY_100', 'CORE_GROWTH', 'CORE_BALANCED', 'CORE_DEFENSIVE', 'SRS_GLOBAL_EQUITY_100',
                  'DISRUPTIVE_TECHNOLOGY', 'ESG_AND_CLEAN_ENERGY', 'HEALTHCARE_INNOVATION', 'CHINA_GROWTH', 'DOWNSIDE_PROTECTED', 'CUSTOM_USD',
                  'INCOME_PRESERVE', 'INCOME_ENHANCE', 'SRS_INCOME_PRESERVE', 'SRS_INCOME_ENHANCED', 'REIT' ,'REIT_RISK_MANAGED', 'SIFN27D',
                  'CASH_PLUS', 'CASH_PLUS_USD', 'CASH_PLUS_GUARANTEED_SGD', 'SRS_CASH_PLUS']

# all_asset_mapping = df_sec_list.merge(syfe_port_mapping, how='left', left_on='Ticker', right_on='internal_port_name')
# all_asset_mapping.loc[all_asset_mapping['MB_port_name'].isnull(), ['MB_port_name', 'internal_port_name']] = all_asset_mapping['Ticker']

In [42]:
# @title 2. Upload MB report

#####################################################################################################################

bbg_last_update_date = pd.read_csv(data_path + 'PV_daily_ret.csv', index_col=0, usecols=[0], parse_dates=['Date'],dayfirst=True).index[-1]
bbg_last_update_date = bbg_last_update_date.strftime('%Y-%m-%d')

#####################################################################################################################
print('\n')
print_arial_bold('Upload client holdings from <a href="https://metabase.internal.syfe.com/question/1470-find-clients-portfolios-by-email?email=&portfolio_id=&client_id=&phone=&source_of_funds=" target="_blank">Metabase</a>')

uploader = widgets.FileUpload(
    accept='*',  # Accepts all file types
    multiple=False  # Only allows one file to be uploaded at a time
)

# function to upload file
def handle_upload(change, syfe_port_mapping=syfe_port_mapping):
    global df_client_portfolios, aggregated_holdings, custom_usd_rows, aggregated_holdings_all_clients, custom_usd_all_clients

    syfe_port_mapping = pd.DataFrame({
    'MB_port_name': ['CASH_PLUS', 'CASH_PLUS_USD', 'CHINA_GROWTH', 'CORE_BALANCED', 'CORE_DEFENSIVE', 'CORE_GROWTH', 'DISRUPTIVE_TECHNOLOGY', 'ESG_AND_CLEAN_ENERGY',
                      'GLOBAL_EQUITY_100', 'HEALTHCARE_INNOVATION', 'INCOME_ENHANCE', 'INCOME_PRESERVE', 'REIT', 'REIT_RISK_MANAGED',
                      'SRS_CASH_PLUS', 'SRS_GLOBAL_EQUITY_100', 'SRS_INCOME_ENHANCED', 'SRS_INCOME_PRESERVE', 'DOWNSIDE_PROTECTED', 'CASH_PLUS_GUARANTEED_SGD',
                    'CUSTOM_USD', 'SIFN27D'],

    'internal_port_name': ['Cash SGD flexi', 'Cash USD flexi', 'China Growth', 'Core Balanced', 'Core Defensive', 'Core Growth', 'Disruptive Technology',
                            'ESG & Clean Energy', 'Core Equity100', 'Healthcare Innovation', 'Income Enhance', 'Income Preserve', 'REIT 100', 'REIT Risk Managed',
                            'SRS Cash SGD flexi', 'SRS Equity100', 'SRS Income Enhance', 'SRS Income Preserve', 'Downside protected', 'Cash SGD guaranteed',
                          'Custom USD', 'Silverdale Fund (Nov 2027)']
    })

    uploaded_filename = next(iter(uploader.value))
    content = uploader.value[uploaded_filename]['content']
    with open(uploaded_filename, 'wb') as f:
        f.write(content)
    print(f'Uploaded `{uploaded_filename}` successfully!')

    # Read the CSV file into a pandas DataFrame
    try:
        df_client_portfolios = pd.read_csv(io.BytesIO(content))
        client_id_list = list((df_client_portfolios['client_id']).unique())
        aggregated_holdings_all_clients = pd.DataFrame()
        custom_usd_all_clients = pd.DataFrame()

        for client_id in client_id_list:
          df_client_portfolios_active = df_client_portfolios[df_client_portfolios['client_id']==client_id]
          df_client_portfolios_active = df_client_portfolios_active[df_client_portfolios_active['status']=='ACTIVE'] # filter for active portfolios only
          if df_client_portfolios_active['actual_holding_weightage'].astype(str).str.startswith('SIFN27D', na=False).any():
              df_client_portfolios_active = process_silverdale_port(df_client_portfolios_active)
          df_client_portfolios_active = process_custom_usd(df_client_portfolios_active) # explode custom usd underlying into individual rows
          # display(df_client_portfolios_active)
          df_client_portfolios_active = df_client_portfolios_active.merge(syfe_port_mapping, how='left', left_on='type', right_on='MB_port_name')

          # Handle custom usd mapping: filter for rows with NaN MB_port_name and update MB_port_name with 'type'
          nan_mb_port_name_rows = df_client_portfolios_active['MB_port_name'].isna()
          df_client_portfolios_active.loc[nan_mb_port_name_rows, 'MB_port_name'] = df_client_portfolios_active.loc[nan_mb_port_name_rows, 'type']
          df_client_portfolios_active.loc[nan_mb_port_name_rows, 'internal_port_name'] = df_client_portfolios_active.loc[nan_mb_port_name_rows, 'type']
          custom_usd_rows = df_client_portfolios_active[nan_mb_port_name_rows].copy()
          custom_usd_rows = custom_usd_rows.merge(df_sec_list[['Ticker']], how='left', left_on='MB_port_name', right_on='Ticker')

          non_custom_usd_rows = df_client_portfolios_active[~nan_mb_port_name_rows]  # Rows that were not originally NaN
          df_client_portfolios_active = pd.concat([non_custom_usd_rows, custom_usd_rows], ignore_index=True)
          # display(df_client_portfolios_active)

          aggregated_holdings = aggregate_client_holdings(df_client_portfolios_active)
          aggregated_holdings['client_id'] = client_id

          aggregated_holdings_all_clients = pd.concat([aggregated_holdings_all_clients, aggregated_holdings],axis=0)
          custom_usd_all_clients = pd.concat([custom_usd_all_clients, custom_usd_rows],axis=0)

        return aggregated_holdings_all_clients

    except pd.errors.ParserError:
        print("Error: Could not parse the file as a CSV. Please ensure it's a valid CSV file.")

uploader.observe(handle_upload, names='value') # Observe the uploader for changes
display(uploader) # Display the uploader

def aggregate_client_holdings(df_client_portfolios):
    # Aggregate holdings based on 'internal_port_name'
    aggregated_holdings = df_client_portfolios.groupby('internal_port_name').agg({
        'internal_port_name': lambda x: ', '.join(x.astype(str)),  # Join tickers with commas
        'nav_in_sgd': 'sum'  # Sum market values
    })
    # aggregated_holdings[['nav_in_sgd']] = aggregated_holdings[['nav_in_sgd']].apply(pd.to_numeric)
    # Calculate allocations
    total_market_value = aggregated_holdings['nav_in_sgd'].sum()
    aggregated_holdings['Allocation'] = aggregated_holdings['nav_in_sgd'] / total_market_value
    aggregated_holdings['Allocation'] = round(aggregated_holdings['Allocation'], 8)
    return aggregated_holdings

def process_custom_usd(df):
    df["nav_in_sgd"] = df["nav_in_sgd"].replace(",", "", regex=True)
    df["nav_in_sgd"] = pd.to_numeric(df["nav_in_sgd"], errors="coerce")
    df["nav_in_usd"] = df["nav_in_usd"].replace(",", "", regex=True)
    df["nav_in_usd"] = pd.to_numeric(df["nav_in_usd"], errors="coerce")

    # Separate CUSTOM_USD portfolios
    custom_usd_df = df[df["type"] == "CUSTOM_USD"].copy()
    other_portfolios_df = df[df["type"] != "CUSTOM_USD"]

    split_custom_usd_df = []

    for _, row in custom_usd_df.iterrows():
        holdings_str = row["actual_holding_weightage"]

        if isinstance(holdings_str, str):  # Ensure it's a string
            try:
                # Split by comma and then by spaces
                holdings = [
                    h.strip().rsplit(" ", 1)  # Split ticker and weight
                    for h in holdings_str.split(",")
                    if h.strip()]
                # print(row)
                for ticker, weight_str in holdings:
                    try:
                        weight = float(weight_str.strip("%")) / 100
                        new_row = row.copy()
                        new_row["type"] = ticker
                        nav_in_sgd = row["nav_in_sgd"]
                        if isinstance(nav_in_sgd, str):
                          nav_in_sgd = float(nav_in_sgd.replace(",", ""))
                        new_row["nav_in_sgd"] = nav_in_sgd * weight if nav_in_sgd else None
                        nav_in_usd = row["nav_in_usd"]
                        if isinstance(nav_in_usd, str):
                            nav_in_usd = float(nav_in_usd.replace(",", ""))
                        new_row["nav_in_usd"] = nav_in_usd * weight if nav_in_usd else None

                        split_custom_usd_df.append(new_row)
                    except ValueError:
                        print(f"Warning: Invalid weight format for ticker {ticker}: {weight_str}")
            except Exception as e:
                print(f"Error processing holdings string '{holdings_str}': {e}")
        else:
            print(f"Warning: actual_holding_weightage is not a string: {holdings_str}")

    # Concatenate the split CUSTOM_USD rows with the non-CUSTOM_USD rows
    processed_df = pd.concat([other_portfolios_df, pd.DataFrame(split_custom_usd_df)],ignore_index=True,)
    processed_df[['nav_in_sgd']] = processed_df[['nav_in_sgd']].apply(pd.to_numeric)
    processed_df[['nav_in_usd']] = processed_df[['nav_in_usd']].apply(pd.to_numeric)


    return processed_df


#####################################################################################################################
# @title Confirm portfolio parameters

def get_unique_tickers():
        all_tickers_set = set(bm_tickers)  # Start with benchmark tickers
        for portfolio_widgets_dict in portfolio_widgets.values():
            portfolio_tickers = re.split(r'\s*,\s*', portfolio_widgets_dict['tickers'].value)
            all_tickers_set.update(portfolio_tickers)  # Add portfolio tickers
        return list(all_tickers_set)

HTML(value='<div style=\'font-family: Arial, sans-serif; font-weight: bold;\'>Upload client holdings from <a h…

FileUpload(value={}, accept='*', description='Upload')

Uploaded `Client_2_PV.csv` successfully!


In [58]:
# @title Calc & Generate PDF

client_id_list = list((df_client_portfolios['client_id']).unique())

for client_id in client_id_list:
  print('Generating report for ', client_id)

  # GENERATE KEY HOLDINGS & ALLOCATIONS

  ###################################################################################################################################################################################################
  # client_latest_alloc = df_client_portfolios[df_client_portfolios['status']=='ACTIVE'][['type', 'nav_in_sgd']]
  client_latest_alloc = aggregated_holdings_all_clients[aggregated_holdings_all_clients['client_id']==client_id]
  client_latest_alloc['nav_in_sgd'] = client_latest_alloc['nav_in_sgd'].astype(str).str.replace(',', '').astype(float)
  # client_latest_alloc = pd.DataFrame(client_latest_alloc.groupby('type')['nav_in_sgd'].sum())
  client_latest_alloc['Client Portfolio'] = client_latest_alloc['nav_in_sgd'] / client_latest_alloc['nav_in_sgd'].sum()
  client_latest_alloc = client_latest_alloc[['Client Portfolio']]
  client_latest_alloc = client_latest_alloc.rename(index=syfe_port_mapping.set_index('MB_port_name')['internal_port_name'])

  client_latest_alloc_adj = client_latest_alloc.T
  client_latest_alloc_adj['reb_flag'] = True

  # split into growth and income portfolios
  all_port_wgt_income_preservation = client_latest_alloc_adj.loc[client_latest_alloc_adj[('reb_flag')] == True].head().iloc[-1:]
  cols_to_keep_income_pres = ['reb_flag'] + [col for col in income_preservation_groups if col in all_port_wgt_income_preservation.columns]
  all_port_wgt_income_preservation = all_port_wgt_income_preservation[cols_to_keep_income_pres]

  all_port_wgt_growth = client_latest_alloc_adj.loc[client_latest_alloc_adj[('reb_flag')] == True].head().iloc[-1:]
  cols_to_keep_growth = ['reb_flag'] + [col for col in all_port_wgt_growth.columns if col != 'reb_flag' and col not in income_preservation_groups]
  # cols_to_keep_growth = ['reb_flag'] + [col for col in growth_groups if col in all_port_wgt_growth.columns]
  all_port_wgt_growth = all_port_wgt_growth[cols_to_keep_growth]
  ###################################################################################################################################################################################################

  assetclass_breakdown_all = pc.exposure_analysis(client_latest_alloc_adj, df_asset_class, latest_data=True)
  assetclass_breakdown_all = (assetclass_breakdown_all.loc[(assetclass_breakdown_all != 0).any(axis=1)].sort_values(by=assetclass_breakdown_all.columns[0], ascending=False))

  country_breakdown_all = pc.exposure_analysis(client_latest_alloc_adj, df_country, latest_data=True)
  country_breakdown_all = (country_breakdown_all.loc[(country_breakdown_all != 0).any(axis=1)].sort_values(by=country_breakdown_all.columns[0], ascending=False))
  country_breakdown_all.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage)

  # growth exposure
  if all_port_wgt_growth.empty == False:
    assetclass_breakdown_growth = pc.exposure_analysis(all_port_wgt_growth, df_asset_class, latest_data=True)
    assetclass_breakdown_growth = (assetclass_breakdown_growth.loc[(assetclass_breakdown_growth != 0).any(axis=1)].sort_values(by=assetclass_breakdown_growth.columns[0], ascending=False))
    assetclass_breakdown_growth = assetclass_breakdown_growth / assetclass_breakdown_growth.sum()

    country_breakdown_growth = pc.exposure_analysis(all_port_wgt_growth, df_country, latest_data=True)
    country_breakdown_growth = (country_breakdown_growth.loc[(country_breakdown_growth != 0).any(axis=1)].sort_values(by=country_breakdown_growth.columns[0], ascending=False))
    country_breakdown_growth.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage)
    country_breakdown_growth = country_breakdown_growth / country_breakdown_growth.sum()

    sector_breakdown_growth = pc.exposure_analysis(all_port_wgt_growth, df_sector, latest_data=True)
    sector_breakdown_growth = (sector_breakdown_growth.loc[(sector_breakdown_growth != 0).any(axis=1)].sort_values(by=sector_breakdown_growth.columns[0], ascending=False))
    sector_breakdown_growth.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage)
    sector_breakdown_growth = sector_breakdown_growth / sector_breakdown_growth.sum()

  # income & preservation exposure
  if all_port_wgt_income_preservation.empty == False:
    assetclass_breakdown_income_preservation = pc.exposure_analysis(all_port_wgt_income_preservation, df_asset_class, latest_data=True)
    assetclass_breakdown_income_preservation = (assetclass_breakdown_income_preservation.loc[(assetclass_breakdown_income_preservation != 0).any(axis=1)].sort_values(by=assetclass_breakdown_income_preservation.columns[0], ascending=False))
    assetclass_breakdown_income_preservation = assetclass_breakdown_income_preservation / assetclass_breakdown_income_preservation.sum()

    country_breakdown_income_preservation = pc.exposure_analysis(all_port_wgt_income_preservation, df_country, latest_data=True)
    country_breakdown_income_preservation = (country_breakdown_income_preservation.loc[(country_breakdown_income_preservation != 0).any(axis=1)].sort_values(by=country_breakdown_income_preservation.columns[0], ascending=False))
    country_breakdown_income_preservation.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage)
    country_breakdown_income_preservation = country_breakdown_income_preservation / country_breakdown_income_preservation.sum()

    sector_breakdown_income_preservation = pc.exposure_analysis(all_port_wgt_income_preservation, df_sector, latest_data=True)
    sector_breakdown_income_preservation = (sector_breakdown_income_preservation.loc[(sector_breakdown_income_preservation != 0).any(axis=1)].sort_values(by=sector_breakdown_income_preservation.columns[0], ascending=False))
    sector_breakdown_income_preservation.pipe(pc.apply_style_heatmap).pipe(pc.apply_2dp_percentage)
    sector_breakdown_income_preservation = sector_breakdown_income_preservation / sector_breakdown_income_preservation.sum()

  ###################################################################################################################################################################################################

  # PORTFOLIO SUMMARY TABLE
  for col in ['nav_in_sgd', 'nav_in_usd', 'pnl_inception', 'invested_amount', 'dividend_balance']:
      df_client_portfolios[col] = pd.to_numeric(df_client_portfolios[col].astype(str).str.replace(',', '', regex=True), errors='coerce')

  # df_client_portfolios = df_client_portfolios.sort_values(by='nav_in_sgd', ascending=False)
  df_client_portfolios_to_show = df_client_portfolios[df_client_portfolios['client_id']==client_id]
  df_client_portfolios_to_show = df_client_portfolios_to_show[df_client_portfolios_to_show['status'] == 'ACTIVE']
  if df_client_portfolios_to_show['actual_holding_weightage'].astype(str).str.startswith('SIFN27D', na=False).any():
      df_client_portfolios_to_show = process_silverdale_port(df_client_portfolios_to_show)
  # Select and format columns
  cols_to_show_all = ['portfolio_id', 'type', 'activated_date_sgt', 'nav_in_sgd', 'nav_in_usd', 'dividend_balance', 'dividend_option', 'maturity_period', 'fd_created_at_rate']
  client_port_rename_cols_all = ['Portfolio ID', 'Portfolio Type', 'Activated date', 'Value (SGD)', 'Value (USD)', 'Dividend (SGD)', 'Dividend Option','Maturity period', 'Guaranteed rate']
  df_client_portfolios_to_show_all = df_client_portfolios_to_show[cols_to_show_all]
  df_client_portfolios_to_show_all.columns = client_port_rename_cols_all
  df_client_portfolios_to_show_all = df_client_portfolios_to_show_all.merge(syfe_port_mapping, left_on='Portfolio Type', right_on='MB_port_name', how='left')

  # Check if 'CUSTOM_USD' exists in 'Portfolio Type'
  if 'CUSTOM_USD' in df_client_portfolios_to_show_all['Portfolio Type'].values:
      custom_usd_risk_rating = calc_custom_weighted_risk_rating(custom_usd_all_clients[custom_usd_all_clients['client_id']==client_id], df_sec_list)

      # Iterate through rows with 'CUSTOM_USD'
      for index, row in df_client_portfolios_to_show_all[df_client_portfolios_to_show_all['Portfolio Type'] == 'CUSTOM_USD'].iterrows():
          # Get the Portfolio ID
          portfolio_id = row['Portfolio ID']

          # Find the corresponding Risk rating in custom_usd_risk_rating
          try:
              risk_rating = custom_usd_risk_rating[custom_usd_risk_rating['portfolio_id'] == portfolio_id]['Risk rating'].values[0]
              # Update the Risk rating in df_client_portfolios_to_show_all
              df_client_portfolios_to_show_all.loc[index, 'Risk rating'] = risk_rating
          except IndexError:
              # Handle cases where portfolio_id is not found in custom_usd_risk_rating
              print(f"Warning: Risk rating not found for Portfolio ID: {portfolio_id}")

  ###################################################################################################################################################################################################

  # PORTFOLIO SUMMARY TABLE
  all_portfolios_summary = df_client_portfolios_to_show_all.copy()

  all_portfolios_summary['Allocation'] = all_portfolios_summary['Value (SGD)'] / all_portfolios_summary['Value (SGD)'].sum()

  total_row_all = pd.DataFrame(columns=all_portfolios_summary.columns)

  for col in all_portfolios_summary.columns:
      if col in ['Value (SGD)', 'Value (USD)', 'Allocation']:
          total_row_all.at['Total', col] = all_portfolios_summary[col].sum()
      elif col == 'Risk rating':
          total_row_all.at['Total', col] = np.dot(
              all_portfolios_summary[col].astype(float), all_portfolios_summary['Allocation']
          ) / all_portfolios_summary['Allocation'].sum()
      else:
          total_row_all.at['Total', col] = ''

  total_row_all.iloc[0, 0] = 'Total'

  all_portfolios_summary[['Value (SGD)', 'Value (USD)']] = round(all_portfolios_summary[['Value (SGD)', 'Value (USD)']], 2)
  growth_rows = all_portfolios_summary[all_portfolios_summary['Portfolio Type'].isin(mb_port_names_growth)]
  income_pres_rows = all_portfolios_summary[all_portfolios_summary['Portfolio Type'].isin(mb_port_names_income_preservation)]

  if not growth_rows.empty:
    growth_subtotal = growth_rows.agg({
        'Value (SGD)': 'sum',
        'Value (USD)': 'sum',
        'Allocation': 'sum',
        'Risk rating': lambda x: np.average(x.astype(float), weights=growth_rows.loc[x.index, 'Allocation'])
    })
    growth_subtotal['Portfolio ID'] = 'Growth Portfolios'
  else:
      # Create an empty Series with 0 values for all columns
      growth_subtotal = pd.Series({
          'Value (SGD)': 0,
          'Value (USD)': 0,
          'Allocation': 0,
          'Risk rating': 0
      }, dtype=object)  # Using object dtype to handle mixed types
      growth_subtotal['Portfolio ID'] = 'Growth Portfolios'
      growth_subtotal = growth_subtotal.to_frame().T  # Convert to DataFrame

  # Calculate subtotals for income_preservation_groups
  if not income_pres_rows.empty:  # Only calculate if there are relevant rows
      income_pres_subtotal = income_pres_rows.agg({
          'Value (SGD)': 'sum',
          'Value (USD)': 'sum',
          'Allocation': 'sum',
          'Risk rating': lambda x: np.average(
              x.astype(float),
              weights=income_pres_rows.loc[x.index, 'Allocation']
          ) if not x.empty else np.nan  # Handle empty cases safely
      })

      income_pres_subtotal['Portfolio ID'] = 'Income & Preservation Portfolios'
  else:
    # Create an empty Series with 0 values for all columns
    income_pres_subtotal = pd.Series({
        'Value (SGD)': 0,
        'Value (USD)': 0,
        'Allocation': 0,
        'Risk rating': 0
    }, dtype=object)  # Using object dtype to handle mixed types
    income_pres_subtotal['Portfolio ID'] = 'Income & Preservation Portfolios'
    income_pres_subtotal = income_pres_subtotal.to_frame().T  # Convert to DataFrame

  # Concatenate subtotals and original DataFrame
  if total_row_all['Value (SGD)'].values[0] > 0:  # Check if NAV (SGD) is greater than 0
      all_portfolios_summary_final = pd.concat([all_portfolios_summary, total_row_all])

  all_portfolios_summary_final['Portfolio Type'] = pd.Categorical(
      all_portfolios_summary_final['Portfolio Type'],
      categories=order_of_ports,
      ordered=True
  )

  # Sort the DataFrame
  all_portfolios_summary_final = all_portfolios_summary_final.sort_values(by='Portfolio Type').reset_index(drop=True)

  other_rows = all_portfolios_summary_final[~all_portfolios_summary_final['L1_classification'].isin(['Growth', 'Income & Preservation'])]

  concat_list = [growth_rows]

  if not growth_rows.empty:
      concat_list.append(growth_subtotal.to_frame().T)

  concat_list.append(income_pres_rows)

  if not income_pres_rows.empty:  # Check if subtotal exists
      concat_list.append(income_pres_subtotal.to_frame().T)

  concat_list.append(other_rows)

  all_portfolios_summary_final = pd.concat(concat_list, ignore_index=True)

  all_portfolios_summary_final[['Value (SGD)', 'Value (USD)']] = all_portfolios_summary_final[['Value (SGD)', 'Value (USD)']].applymap(lambda x: "{:,.2f}".format(x) if isinstance(x, (int, float)) else x)
  all_portfolios_summary_final[['Risk rating']] = all_portfolios_summary_final[['Risk rating']].applymap(lambda x: "{:,.1f}".format(x) if isinstance(x, (int, float)) else x)
  all_portfolios_summary_final['Allocation'] = (all_portfolios_summary_final['Allocation'] * 100).map('{:.2f}%'.format)

  for col in all_portfolios_summary_final.columns:
      if pd.api.types.is_categorical_dtype(all_portfolios_summary_final[col]):
          # Add an empty string to the categories
          all_portfolios_summary_final[col] = all_portfolios_summary_final[col].cat.add_categories([''])
          # Fill NaN with the empty string
          all_portfolios_summary_final[col] = all_portfolios_summary_final[col].fillna('')

      elif all_portfolios_summary_final[col].dtype == 'object':  # For string columns
          all_portfolios_summary_final[col] = all_portfolios_summary_final[col].fillna('')

      else:  # For numeric columns
          all_portfolios_summary_final[col] = all_portfolios_summary_final[col].fillna(0)


  all_portfolios_summary_final = all_portfolios_summary_final[['Portfolio ID', 'internal_port_name', 'L2_classification', 'Risk rating', 'Activated date', 'Value (SGD)', 'Value (USD)', 'Allocation']]
  all_portfolios_summary_final.rename(columns={'internal_port_name': 'Portfolio Type', 'L2_classification':'Classification', 'Risk rating':'Risk rating (Max:5)'}, inplace=True)

  ###################################################################################################################################################################################################
  # GROWTH PORTFOLIO TABLE

  # Separate the dataframe based on the 'type' column
  growth_portfolios = all_portfolios_summary[all_portfolios_summary['L1_classification']=='Growth']
  growth_portfolios['Allocation'] = growth_portfolios['Value (SGD)'] / growth_portfolios['Value (SGD)'].sum()

  core_subtotal = growth_portfolios[growth_portfolios['L2_classification']=='Core'].sum(numeric_only=True)
  core_subtotal['Portfolio ID'] = 'Core'
  specialised_subtotal = growth_portfolios[growth_portfolios['L2_classification']=='Specialised'].sum(numeric_only=True)
  specialised_subtotal['Portfolio ID'] = 'Specialised'

  total_row_growth = pd.DataFrame({col: [growth_portfolios[col].sum() if col in ['Value (SGD)', 'Value (USD)', 'Invested Amount', 'P&L since inception', 'Dividend (SGD)', 'Allocation'] else '']
                            for col in growth_portfolios.columns}, index=['Total'])

  total_row_growth.iloc[0, 0] = 'Total'

  if total_row_growth['Value (SGD)'].values[0] > 0:  # Check if NAV (SGD) is greater than 0
      growth_portfolios = pd.concat([growth_portfolios, total_row_growth])

  growth_portfolios['Portfolio Type'] = pd.Categorical(
      growth_portfolios['Portfolio Type'],
      categories=order_of_ports,
      ordered=True
  )

  # Sort the DataFrame
  growth_portfolios = growth_portfolios.sort_values(by='Portfolio Type').reset_index(drop=True)
  core_rows = growth_portfolios[growth_portfolios['L2_classification'] == 'Core']
  specialised_rows = growth_portfolios[growth_portfolios['L2_classification'] == 'Specialised']
  other_rows = growth_portfolios[~growth_portfolios['L2_classification'].isin(['Core', 'Specialised'])]

  # growth_portfolios = pd.concat([core_rows, core_subtotal.to_frame().T, specialised_rows, specialised_subtotal.to_frame().T, other_rows])

  growth_portfolios_list = []  # Start with an empty list
  growth_portfolios_list.append(core_rows)

  if core_rows['Value (SGD)'].sum() != 0:
      growth_portfolios_list.append(core_subtotal.to_frame().T)

  growth_portfolios_list.append(specialised_rows)

  if specialised_rows['Value (SGD)'].sum() != 0:
      growth_portfolios_list.append(specialised_subtotal.to_frame().T)

  growth_portfolios_list.append(other_rows)

  # Concatenate the list of DataFrames
  growth_portfolios = pd.concat(growth_portfolios_list)

  growth_portfolios[['Value (SGD)', 'Value (USD)']] = growth_portfolios[['Value (SGD)', 'Value (USD)']].applymap(lambda x: "{:,.2f}".format(x) if isinstance(x, (int, float)) else x)
  growth_portfolios['Allocation'] = (growth_portfolios['Allocation'] * 100).map('{:.2f}%'.format)

  for col in growth_portfolios.columns:
      if pd.api.types.is_categorical_dtype(growth_portfolios[col]):
          # Add an empty string to the categories
          growth_portfolios[col] = growth_portfolios[col].cat.add_categories([''])
          # Fill NaN with the empty string
          growth_portfolios[col] = growth_portfolios[col].fillna('')

      elif growth_portfolios[col].dtype == 'object':  # For string columns
          growth_portfolios[col] = growth_portfolios[col].fillna('')

      else:  # For numeric columns
          growth_portfolios[col] = growth_portfolios[col].fillna(0)

  growth_portfolios = growth_portfolios[['Portfolio ID', 'internal_port_name', 'Value (SGD)', 'Value (USD)',  'Allocation']]
  growth_portfolios.rename(columns={'internal_port_name': 'Portfolio Type'}, inplace=True)

  ###################################################################################################################################################################################################
  # INCOME & PRESERVATION PORTFOLIOS TABLE

  income_preservation_fi_metrics = df_fi_metrics[[ticker for ticker in df_fi_metrics.columns if ticker in income_preservation_groups]]
  # Create a dictionary for ticker to MB_port_name mapping
  ticker_to_mb_mapping = dict(zip(syfe_port_mapping['internal_port_name'], syfe_port_mapping['MB_port_name']))
  income_preservation_fi_metrics = income_preservation_fi_metrics.rename(columns=ticker_to_mb_mapping)
  income_preservation_fi_metrics.T[['YTM / est. yield']]

  income_pres_portfolios = all_portfolios_summary[all_portfolios_summary['L1_classification']=='Income & Preservation']
  income_pres_portfolios = pd.merge(income_pres_portfolios, income_preservation_fi_metrics.T[['YTM / est. yield']], left_on='Portfolio Type', right_index=True, how='left')
  income_pres_portfolios['Est. Yield (%)'] = income_pres_portfolios.apply(lambda row: row['YTM / est. yield'] if row['Portfolio Type'] != 'CASH_PLUS_GUARANTEED_SGD' else row['Guaranteed rate'], axis=1)
  income_pres_portfolios = income_pres_portfolios.drop(columns=['YTM / est. yield', 'Guaranteed rate'])

  # Ensure correct data types
  income_pres_portfolios['Allocation'] = pd.to_numeric(income_pres_portfolios['Value (SGD)'], errors='coerce') / income_pres_portfolios['Value (SGD)'].sum()
  income_pres_portfolios['Est. Yield (%)'] = pd.to_numeric(income_pres_portfolios['Est. Yield (%)'], errors='coerce')
  income_pres_portfolios['Allocation'] = pd.to_numeric(income_pres_portfolios['Allocation'], errors='coerce')
  income_pres_portfolios['Proj. Dividend (SGD)'] = income_pres_portfolios['Est. Yield (%)'] / 100 * income_pres_portfolios['Value (SGD)']

  # Calculate subtotals
  income_plus_subtotal = calculate_subtotal_income_preservation(income_pres_portfolios, 'Income+')
  reits_subtotal = calculate_subtotal_income_preservation(income_pres_portfolios, 'REITs')
  alt_subtotal = calculate_subtotal_income_preservation(income_pres_portfolios, 'Alternatives')
  cash_mmf_subtotal = calculate_subtotal_income_preservation(income_pres_portfolios, 'Cash / MMF')

  income_pres_portfolios['Est. Yield (%)'] = pd.to_numeric(income_pres_portfolios['Est. Yield (%)'], errors='coerce')
  income_pres_portfolios['Proj. Dividend (SGD)'] = pd.to_numeric(income_pres_portfolios['Proj. Dividend (SGD)'], errors='coerce')

  total_row_income_pres = pd.DataFrame({
      col: [
          # Sum numeric columns
          income_pres_portfolios[col].sum() if col in ['Value (SGD)', 'Value (USD)', 'Invested Amount', 'P&L since inception', 'Dividend (SGD)', 'Allocation', 'Proj. Dividend (SGD)'] else
          # Calculate weighted average for Yield (%) column safely
          ((income_pres_portfolios[col] * income_pres_portfolios['Allocation']).sum() / income_pres_portfolios['Allocation'].sum())
          if col == 'Est. Yield (%)' and income_pres_portfolios['Allocation'].sum() != 0 else
          ''  # Keep other columns empty
      ] for col in income_pres_portfolios.columns
  }, index=['Total'])

  total_row_income_pres.iloc[0, 0] = 'Total'

  if total_row_income_pres['Value (SGD)'].values[0] > 0:  # Check if NAV (SGD) is greater than 0
      income_pres_portfolios = pd.concat([income_pres_portfolios, total_row_income_pres])

  income_pres_portfolios['Portfolio Type'] = pd.Categorical(
      income_pres_portfolios['Portfolio Type'],
      categories=order_of_ports,
      ordered=True
  )

  # Sort the DataFrame
  income_pres_portfolios = income_pres_portfolios.sort_values(by='Portfolio Type').reset_index(drop=True)
  income_plus_rows = income_pres_portfolios[income_pres_portfolios['L2_classification'] == 'Income+']
  reits_rows = income_pres_portfolios[income_pres_portfolios['L2_classification'] == 'REITs']
  alt_rows = income_pres_portfolios[income_pres_portfolios['L2_classification'] == 'Alternatives']
  cm_rows = income_pres_portfolios[income_pres_portfolios['L2_classification'] == 'Cash / MMF']
  others_rows = income_pres_portfolios[~income_pres_portfolios['L2_classification'].isin(['Income+', 'REITs', 'Alternatives', 'Cash / MMF'])]


  # income_pres_portfolios = pd.concat([income_plus_rows, income_plus_subtotal.to_frame().T, reits_rows, reits_subtotal.to_frame().T, cm_rows, cash_mmf_subtotal.to_frame().T, total_row_income_pres])

  income_pres_portfolios_list = []  # Start with an empty list
  income_pres_portfolios_list.append(income_plus_rows)

  if income_plus_rows['Value (SGD)'].sum() != 0:
      income_pres_portfolios_list.append(income_plus_subtotal.to_frame().T)

  income_pres_portfolios_list.append(reits_rows)

  if reits_rows['Value (SGD)'].sum() != 0:
      income_pres_portfolios_list.append(reits_subtotal.to_frame().T)

  income_pres_portfolios_list.append(alt_rows)

  if alt_rows['Value (SGD)'].sum() != 0:
      income_pres_portfolios_list.append(alt_subtotal.to_frame().T)

  income_pres_portfolios_list.append(cm_rows)

  if cm_rows['Value (SGD)'].sum() != 0:
      income_pres_portfolios_list.append(cash_mmf_subtotal.to_frame().T)

  income_pres_portfolios_list.append(total_row_income_pres)

  # Concatenate the list of DataFrames
  income_pres_portfolios = pd.concat(income_pres_portfolios_list)

  income_pres_portfolios[['Value (SGD)', 'Value (USD)', 'Est. Yield (%)', 'Dividend (SGD)', 'Proj. Dividend (SGD)']] = income_pres_portfolios[['Value (SGD)', 'Value (USD)', 'Est. Yield (%)', 'Dividend (SGD)', 'Proj. Dividend (SGD)']].applymap(lambda x: "{:,.2f}".format(x) if isinstance(x, (int, float)) else x)
  income_pres_portfolios['Allocation'] = (income_pres_portfolios['Allocation'] * 100).map('{:.2f}%'.format)

  for col in income_pres_portfolios.columns:
      if pd.api.types.is_categorical_dtype(income_pres_portfolios[col]):
          # Add an empty string to the categories
          income_pres_portfolios[col] = income_pres_portfolios[col].cat.add_categories([''])
          # Fill NaN with the empty string
          income_pres_portfolios[col] = income_pres_portfolios[col].fillna('')

      elif income_pres_portfolios[col].dtype == 'object':  # For string columns
          income_pres_portfolios[col] = income_pres_portfolios[col].fillna('')

      else:  # For numeric columns
          income_pres_portfolios[col] = income_pres_portfolios[col].fillna(0)

  income_pres_portfolios['Dividend Option adj'] = income_pres_portfolios.apply(dividend_option_adj, axis=1)
  income_pres_portfolios.drop(columns=['Dividend Option'], inplace=True)

  # Apply the function to the 'Maturity Period' column
  income_pres_portfolios['Maturity period'] = income_pres_portfolios['Maturity period'].apply(format_maturity_period)

  income_pres_portfolios = income_pres_portfolios[['Portfolio ID', 'internal_port_name', 'Value (SGD)', 'Value (USD)', 'Est. Yield (%)', 'Proj. Dividend (SGD)', 'Dividend Option adj','Maturity period', 'Allocation']]
  income_pres_portfolios.rename(columns={'internal_port_name': 'Portfolio Type', 'Est. Yield (%)':'Est. Yield (%)*', 'Proj. Dividend (SGD)':'Proj. Dividend (SGD)**', 'Dividend Option adj':'Dividend Option'}, inplace=True)

  ###################################################################################################################################################################################################

  # GENERATE PDF

  client_name = 'CLIENT NAME'
  template_pdf = PyPDF2.PdfReader(data_path + 'Portfolio_Visualiser_Template_Landscape.pdf')

  # if client_latest_alloc[i][client_latest_alloc[i].index.isin(growth_groups)].empty == False:
  #   growth_breakdown_chart = plot_sunburst_chart_growth(client_latest_alloc[i][client_latest_alloc[i].index.isin(growth_groups)], title='Growth Allocation')

  # if client_latest_alloc[i][client_latest_alloc[i].index.isin(income_preservation_groups)].empty == False:
  #   income_pres_breakdown_chart = plot_sunburst_chart_income_preservation(client_latest_alloc[i][client_latest_alloc[i].index.isin(income_preservation_groups)],
  #                                                               title='Income & Preservation Allocation')
  # if client_latest_alloc[i][client_latest_alloc[i].index.isin(income_preservation_groups)].empty == False and client_latest_alloc[i][client_latest_alloc[i].index.isin(growth_groups)].empty == False:
  #   growth_income_breakdown_chart = plot_sunburst_chart_growth_income(client_latest_alloc[i], title='Growth & Income Allocation')

  top_asset_class_chart_all = plot_pie_chart_topX(assetclass_breakdown_all.iloc[:, 0], title='Asset Class Exposure', colors=syfe_colors)
  top_asset_class_chart_growth = plot_pie_chart_topX(assetclass_breakdown_growth.iloc[:, 0], title='Asset Class Exposure', colors=syfe_colors)
  top_asset_class_chart_income_preservation = plot_pie_chart_topX(assetclass_breakdown_income_preservation.iloc[:, 0], title='Asset Class Exposure', colors=syfe_colors)

  top_country_chart_all = plot_bar_chart_topX(country_breakdown_all.iloc[:, 0], title="Country Exposure",  colors=syfe_colors)
  top_country_chart_growth = plot_bar_chart_topX(country_breakdown_growth.iloc[:, 0], title="Country Exposure",  colors=syfe_colors)
  top_country_chart_income_preservation = plot_bar_chart_topX(country_breakdown_income_preservation.iloc[:, 0], title="Country Exposure",  colors=syfe_colors)

  top_sector_chart_growth = plot_bar_chart_topX(sector_breakdown_growth.iloc[:, 0], title="Sector Exposure",  colors=syfe_colors)
  top_sector_chart_income_preservation = plot_bar_chart_topX(sector_breakdown_income_preservation.iloc[:, 0], title="Sector Exposure", colors=syfe_colors)

  # CREATE INDIVIDUAL PDF & MERGE PDFS
  # Assuming you have your table DataFrame, title, and chart variables
  create_pdf_with_table_and_charts("portfolio_summary.pdf", all_portfolios_summary_final, "Portfolio Summary",[top_asset_class_chart_all, top_country_chart_all],
                                  col_width_manual=[145, 125, 70, 110, 90, 90, 90, 69.7])

  if not growth_portfolios.empty:
      create_pdf_with_table_and_charts("growth_portfolios.pdf", growth_portfolios, "Growth Portfolios", [top_asset_class_chart_growth, top_country_chart_growth, top_sector_chart_growth])

  if not income_pres_portfolios.empty:
      create_pdf_with_table_and_charts("income_preservation_portfolios.pdf", income_pres_portfolios, "Income & Preservation Portfolios",
                                      [top_asset_class_chart_income_preservation , top_country_chart_income_preservation, top_sector_chart_income_preservation],
                                      col_width_manual=[93, 120, 80, 80, 80, 100, 90, 100, 50])

  # Merge PDFs
  appendix_pdf = PyPDF2.PdfReader(data_path + 'Portfolio_Visualiser_Appendix.pdf')

  merger = PdfMerger()
  merger.append("portfolio_summary.pdf")
  if not growth_portfolios.empty:
      merger.append("growth_portfolios.pdf")
  if not income_pres_portfolios.empty:
      merger.append("income_preservation_portfolios.pdf")
  merger.write(str(client_id) + "_report_merged.pdf")

  # Paths
  template_pdf_path = os.path.join(data_path, "Portfolio_Visualiser_Template_Landscape.pdf")
  cover_pdf_path = os.path.join(data_path, "Portfolio_Visualiser_Cover.pdf")
  overlay_pdf_path = os.path.join('/content/Projects/' + str(client_id) + '_report_merged.pdf')
  output_pdf_path = os.path.join('/content/Projects/' + str(client_id) + '_report_final.pdf')

  # Merge overlay onto the template PDF
  merge_with_template(template_pdf_path, overlay_pdf_path, output_pdf_path)

  merger = PdfMerger()
  merger.append(output_pdf_path)
  # merger.append(appendix_pdf)
  merger.write(output_pdf_path)

  add_page_numbers_header(output_pdf_path, output_pdf_path, client_name=client_name)
  merger = PdfMerger()
  merger.append(cover_pdf_path)
  merger.append(output_pdf_path)
  merger.write(output_pdf_path)

  files.download(output_pdf_path)

  print(f"Final report generated: {output_pdf_path}")


Generating report for  2


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Final report generated: /content/Projects/2_report_final.pdf


In [57]:
sum([93, 120, 80, 80, 80, 100, 90, 100, 55])

798

In [ ]:
# @title Archive

# Func sunburst charts
# Function to save charts to a single PDF

from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from reportlab.lib.pagesizes import letter, A4, landscape
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, BaseDocTemplate, Frame, PageTemplate, PageBreak
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import cm  # Import cm for specifying width in centimeters


########################################################################################################################

syfe_colors = ['#263159', '#2f51c9', '#879be3', '#bcbed7', '#dedfee','#fff2cc', '#ffe599', '#e3bf61', '#666666', '#7d839b', '#414e7d']

def convert_maturity_period(value):
    """Converts maturity period values to the desired format, handling non-string values."""
    if isinstance(value, str):  # Check if value is a string
        match = re.search(r'(\w+)_MONTHS', value)
        if match:
            period_num = {
                'THREE': '3',
                'SIX': '6',
                'TWELVE': '12'  # Add more mappings as needed
            }.get(match.group(1))
            if period_num:
                return f"{period_num} months"
    return value  # Return original value if not a string or no match

def plot_sunburst_chart_MP(df_allocation, title=None, width=1000, height=600):
    # Define groups
    core_groups = ['Core E100', 'Core Growth', 'Core Balanced', 'Core Defensive', 'SRS E100']
    specialised_groups = ['China Growth', 'Disruptive Tech', 'ESG & Clean Energy', 'Healthcare Innovation', 'Downside protected']
    passive_income_groups = ['Income Enhance', 'Income Preserve', 'REIT 100', 'REIT RM', 'SRS Income Enhance', 'SRS Income Preserve']
    custom_groups = ['CUSTOM_USD']

    # Filter out rows with allocation <= 0.0001 or NaN
    df_allocation = df_allocation[df_allocation != 0].dropna()

    # Categorize portfolios
    categories = []
    for portfolio in df_allocation.index:
        if portfolio in core_groups:
            categories.append('Core')
        elif portfolio in specialised_groups:
            categories.append('Specialised')
        elif portfolio in passive_income_groups:
            categories.append('Passive Income')
        elif portfolio in custom_groups:
            categories.append('Custom')
        else:
            categories.append('Other')

    # Dynamically create the list of unique categories based on the data
    unique_categories = sorted(set(categories))  # Only include categories present in the data
    labels = ['Portfolio Allocation'] + unique_categories + list(df_allocation.index)
    parents = [''] + ['Portfolio Allocation'] * len(unique_categories) + categories

    # Calculate values dynamically for existing categories
    category_sums = []
    for unique_cat in unique_categories:
        category_sum = df_allocation.loc[
            [portfolio for portfolio in df_allocation.index if
             (portfolio in core_groups and unique_cat == 'Core') or
             (portfolio in specialised_groups and unique_cat == 'Specialised') or
             (portfolio in custom_groups and unique_cat == 'Custom') or
             (portfolio in passive_income_groups and unique_cat == 'Passive Income')]
        ].sum()
        category_sums.append(category_sum)

    values = [df_allocation.sum()] + category_sums + list(df_allocation.values)

    # Filter out nodes where values <= 0
    filtered_data = [
        (label, parent, value) for label, parent, value in zip(labels, parents, values) if value > 0
    ]
    filtered_labels = [x[0] for x in filtered_data]
    filtered_parents = [x[1] for x in filtered_data]
    filtered_values = [x[2] for x in filtered_data]

    # Create the sunburst chart
    fig = go.Figure(go.Sunburst(
        labels=filtered_labels,
        parents=filtered_parents,
        values=filtered_values,
        branchvalues="total",
        textinfo="label+percent parent",  # Include labels and percent for all nodes
        insidetextorientation="horizontal",
    ))

    # Customize traces
    fig.update_traces(
        texttemplate=[
            "%{label}<br>%{percentRoot:.2%}" if parent != "" else ""
            for parent in filtered_parents
        ],
        outsidetextfont={"size": 12, "family": "Arial"},
        marker=dict(line=dict(width=0.5, color="white"))
    )

    # Update layout
    fig.update_layout(
        title_text=title,
        margin=dict(t=50, l=0, r=0, b=0),
        width=width,
        height=height,
        uniformtext=dict(minsize=10, mode='show')  # Uniform text size setting
    )
    return fig


def plot_sunburst_chart_MP_and_CM(df_allocation, title=None, width=1000, height=600):
    # Define groups
    core_groups = ['Core E100', 'Core Growth', 'Core Balanced', 'Core Defensive', 'SRS E100']
    specialised_groups = ['China Growth', 'Disruptive Tech', 'ESG & Clean Energy', 'Healthcare Innovation', 'Downside protected']
    passive_income_groups = ['Income Enhance', 'Income Preserve', 'REIT 100', 'REIT RM', 'SRS Income Enhance', 'SRS Income Preserve']
    custom_groups = ['CUSTOM_USD']

    mp_groups = core_groups + specialised_groups + passive_income_groups + custom_groups
    cm_groups = ['Cash SGD flexi', 'Cash USD flexi', 'Cash SGD guaranteed']

    # Filter out rows with allocation of 0 or NaN
    df_allocation = df_allocation[df_allocation != 0].dropna()

    # Categorize portfolios
    categories = []
    for portfolio in df_allocation.index:
        if portfolio in mp_groups:
            categories.append('Managed Portfolios')
        elif portfolio in cm_groups:
            categories.append('Cash Management')
        else:
            categories.append('Other')

    # Create labels, parents, and values
    unique_categories = ['Managed Portfolios', 'Cash Management']
    labels = ['Portfolio Allocation'] + unique_categories + list(df_allocation.index)
    parents = [''] + ['Portfolio Allocation'] * len(unique_categories) + categories

    # Corrected calculation of values
    category_sums = []
    for unique_cat in unique_categories:
        category_sum = df_allocation.loc[[portfolio for portfolio in df_allocation.index if portfolio in mp_groups and unique_cat == 'Managed Portfolios' or \
                                          portfolio in cm_groups and unique_cat == 'Cash Management']].sum()
        category_sums.append(category_sum)

    values = [df_allocation.sum()] + category_sums + list(df_allocation.values)

    # Create the sunburst chart
    fig = go.Figure(go.Sunburst(
    labels=labels,
    parents=parents,
    values=values,
    branchvalues="total",
    textinfo="label+percent parent",  # Include labels and percent for all nodes
    insidetextorientation="horizontal",))

    # Customize traces to hide the root node label
    fig.update_traces(
    texttemplate=[
        "%{label}<br>%{percentRoot:.2%}" if parent != "" else ""
        for parent in parents
    ],
    outsidetextfont={"size": 12},
    marker=dict(line=dict(width=0.5, color="white"))
    )

    fig.update_layout(
        title_text=title,
        margin=dict(t=50, l=0, r=0, b=0),
        width=width,
        height=height,
        uniformtext=dict(minsize=10, mode='show')  # Uniform text size setting
    )
    return fig

def plot_sunburst_chart_growth(df_allocation, title=None, width=1000, height=600):
    # Define groups
    core_groups = ['Core E100', 'Core Growth', 'Core Balanced', 'Core Defensive', 'SRS E100']
    specialised_groups = ['China Growth', 'Disruptive Tech', 'ESG & Clean Energy', 'Healthcare Innovation', 'Downside protected', 'Custom USD']

    # Filter out rows with allocation of 0 or NaN
    df_allocation = df_allocation[df_allocation != 0].dropna()

    # Categorize portfolios
    categories = []
    for portfolio in df_allocation.index:
        if portfolio in core_groups:
            categories.append('Core')
        elif portfolio in specialised_groups:
            categories.append('Specialised')
        else:
            categories.append('Other')

    # Create labels, parents, and values
    unique_categories = ['Core', 'Specialised']
    labels = ['Portfolio Allocation'] + unique_categories + list(df_allocation.index)
    parents = [''] + ['Portfolio Allocation'] * len(unique_categories) + categories

    # Calculate category sums
    category_sums = []
    for unique_cat in unique_categories:
        category_sum = df_allocation.loc[[portfolio for portfolio in df_allocation.index if portfolio in core_groups and unique_cat == 'Core' or \
                                          portfolio in specialised_groups and unique_cat == 'Specialised']].sum()
        category_sums.append(category_sum)

    values = [df_allocation.sum()] + category_sums + list(df_allocation.values)

    # Filter nodes with values > 0
    filtered_data = [
        (label, parent, value) for label, parent, value in zip(labels, parents, values) if value > 0
    ]
    filtered_labels = [x[0] for x in filtered_data]
    filtered_parents = [x[1] for x in filtered_data]
    filtered_values = [x[2] for x in filtered_data]

    # Create the sunburst chart
    fig = go.Figure(go.Sunburst(
        labels=filtered_labels,
        parents=filtered_parents,
        values=filtered_values,
        branchvalues="total",
        textinfo="label+percent parent",  # Include labels and percent for all nodes
        insidetextorientation="horizontal",
    ))

    # Customize traces to hide the root node label
    fig.update_traces(
        texttemplate=[
            "%{label}<br>%{percentRoot:.2%}" if parent != "" else ""
            for parent in filtered_parents
        ],
        outsidetextfont={"size": 12},
        marker=dict(line=dict(width=0.5, color="white"))
    )

    # Update layout
    fig.update_layout(
        title_text=title,
        margin=dict(t=50, l=0, r=0, b=0),
        width=width,
        height=height,
        uniformtext=dict(minsize=10, mode='show')  # Uniform text size setting
    )
    return fig

def plot_sunburst_chart_income_preservation(df_allocation, title=None, width=1000, height=600):
    # Define groups
    income_groups = ['Income Enhance', 'Income Preserve', 'SRS Income Enhance', 'SRS Income Preserve']
    reits_groups = ['REIT 100', 'REIT RM']
    cm_groups = ['Cash SGD flexi', 'Cash USD flexi', 'Cash SGD guaranteed']

    # Filter out rows with allocation of 0 or NaN
    df_allocation = df_allocation[df_allocation != 0].dropna()

    # Categorize portfolios
    categories = []
    for portfolio in df_allocation.index:
        if portfolio in income_groups:
            categories.append('Bonds')
        elif portfolio in reits_groups:
            categories.append('REITs')
        elif portfolio in cm_groups:
            categories.append('Cash / MMF')
        else:
            categories.append('Other')

    # Create labels, parents, and values
    unique_categories = ['Bonds', 'REITs', 'Cash / MMF']
    labels = ['Portfolio Allocation'] + unique_categories + list(df_allocation.index)
    parents = [''] + ['Portfolio Allocation'] * len(unique_categories) + categories

    # Corrected calculation of values
    category_sums = []
    for unique_cat in unique_categories:
        category_sum = df_allocation.loc[[portfolio for portfolio in df_allocation.index if portfolio in income_groups and unique_cat == 'Bonds' or
                                          portfolio in reits_groups and unique_cat == 'REITs' or
                                          portfolio in cm_groups and unique_cat == 'Cash / MMF']].sum()
        category_sums.append(category_sum)

    values = [df_allocation.sum()] + category_sums + list(df_allocation.values)

    # Create the sunburst chart
    fig = go.Figure(go.Sunburst(
    labels=labels,
    parents=parents,
    values=values,
    branchvalues="total",
    textinfo="label+percent parent",  # Include labels and percent for all nodes
    insidetextorientation="horizontal",))

    # Customize traces to hide the root node label
    fig.update_traces(
    texttemplate=[
        "%{label}<br>%{percentRoot:.2%}" if parent != "" else ""
        for parent in parents
    ],
    outsidetextfont={"size": 12},
    marker=dict(line=dict(width=0.5, color="white"))
    )

    fig.update_layout(
        title_text=title,
        margin=dict(t=50, l=0, r=0, b=0),
        width=width,
        height=height,
        uniformtext=dict(minsize=10, mode='show')  # Uniform text size setting
    )
    return fig

def plot_sunburst_chart_growth_income(df_allocation, title=None, width=1000, height=600):
    # Define groups
    growth_groups = ['Core E100', 'Core Growth', 'Core Balanced', 'Core Defensive','SRS E100', 'China Growth','Disruptive Tech', 'ESG & Clean Energy', 'Healthcare Innovation', 'Downside protected', 'Custom USD']
    income_preservation_groups = ['Income Enhance', 'Income Preserve','REIT 100', 'REIT RM', 'SRS Income Enhance','SRS Income Preserve', 'Cash SGD flexi', 'Cash USD flexi', 'Cash SGD guaranteed']

    # Filter out rows with allocation of 0 or NaN
    df_allocation = df_allocation[df_allocation != 0].dropna()

    # Categorize portfolios
    categories = []
    for portfolio in df_allocation.index:
        if portfolio in growth_groups:
            categories.append('Growth')
        elif portfolio in income_preservation_groups:
            categories.append('Income & Preservation')
        else:
            categories.append('Other')

    # Create labels, parents, and values
    unique_categories = ['Growth', 'Income & Preservation']
    labels = ['Portfolio Allocation'] + unique_categories + list(df_allocation.index)
    parents = [''] + ['Portfolio Allocation'] * len(unique_categories) + categories

    # Corrected calculation of values
    category_sums = []
    for unique_cat in unique_categories:
        category_sum = df_allocation.loc[[portfolio for portfolio in df_allocation.index if portfolio in growth_groups and unique_cat == 'Growth' or
                                          portfolio in income_preservation_groups and unique_cat == 'Income & Preservation']].sum()
        category_sums.append(category_sum)

    values = [df_allocation.sum()] + category_sums + list(df_allocation.values)

    # Create the sunburst chart
    fig = go.Figure(go.Sunburst(
    labels=labels,
    parents=parents,
    values=values,
    branchvalues="total",
    textinfo="label+percent parent",  # Include labels and percent for all nodes
    insidetextorientation="horizontal",))

    # Customize traces to hide the root node label
    fig.update_traces(
    texttemplate=[
        "%{label}<br>%{percentRoot:.2%}" if parent != "" else ""
        for parent in parents
    ],
    outsidetextfont={"size": 12},
    marker=dict(line=dict(width=0.5, color="white"))
    )

    fig.update_layout(
        title_text=title,
        margin=dict(t=50, l=0, r=0, b=0),
        width=width,
        height=height,
        uniformtext=dict(minsize=10, mode='show')  # Uniform text size setting
    )
    return fig


# Fixed income metrics - duration, YTM, dividend yield, credit quality

# # calc dividend of all constituents
# dvd_breakdown_all = pc.exposure_analysis(client_latest_alloc_adj, df_fi_metrics.loc[['T12M dividend yield']].astype(float), latest_data=True)
# dvd_breakdown_all = round(dvd_breakdown_all, 2)


# # Duration, YTM and credit quality of FI components only
# fixed_income_list = df_fi_metrics.columns[df_fi_metrics.iloc[df_fi_metrics.index.get_loc('YTM / est. yield')] != 0].tolist()
# port_fi_metrics = pd.DataFrame()

# ytm_duration_df = pc.exposure_analysis_mixed(client_latest_alloc_adj, df_fi_metrics.loc[['Duration', 'YTM / est. yield']].astype(float), fixed_income_list, latest_data=True)
# port_fi_metrics = pd.concat([port_fi_metrics, ytm_duration_df], axis=1)
# port_fi_metrics.loc['Credit rating'] = pc.calc_avg_credit_rating(client_latest_alloc_adj, df_fi_metrics.loc[['Credit rating']], latest_data=True)

# # port_fi_metrics = pd.concat([port_fi_metrics, df_fi_metrics.loc[['Duration', 'YTM / est. yield', 'Credit rating']][bm_tickers]], axis=1)
# port_fi_metrics = pd.concat([port_fi_metrics, dvd_breakdown_all], axis=0)
# port_fi_metrics